# Project
# Srinithish Kandagadla

In [132]:
from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import MinMaxScaler
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
import numpy as np
import random
from pyspark.sql.functions import udf
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
import matplotlib.pyplot as plt
import pyspark
print(pyspark.__version__)
import re
from pyspark.ml.feature import CountVectorizer
import matplotlib.pyplot as plt
from math import log
from pyspark.ml.feature import VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql import SQLContext


2.4.4


In [133]:
sqlContext = SQLContext(spark)

In [122]:
spark = SparkSession.builder.master("local[*]").appName("Project").getOrCreate()    
spark.conf.set("spark.executor.memory", '32g')
spark.conf.set("spark.driver.memory",'32g')

In [61]:
CreditDefaultData = spark.read.csv("default of credit card clients1.csv",header=True,sep=",",inferSchema=True)

In [62]:
CreditDefaultData.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- X1: string (nullable = true)
 |-- X2: string (nullable = true)
 |-- X3: string (nullable = true)
 |-- X4: string (nullable = true)
 |-- X5: string (nullable = true)
 |-- X6: string (nullable = true)
 |-- X7: string (nullable = true)
 |-- X8: string (nullable = true)
 |-- X9: string (nullable = true)
 |-- X10: string (nullable = true)
 |-- X11: string (nullable = true)
 |-- X12: string (nullable = true)
 |-- X13: string (nullable = true)
 |-- X14: string (nullable = true)
 |-- X15: string (nullable = true)
 |-- X16: string (nullable = true)
 |-- X17: string (nullable = true)
 |-- X18: string (nullable = true)
 |-- X19: string (nullable = true)
 |-- X20: string (nullable = true)
 |-- X21: string (nullable = true)
 |-- X22: string (nullable = true)
 |-- X23: string (nullable = true)
 |-- Y: string (nullable = true)



In [63]:
CreditDefaultData.explain()

== Physical Plan ==
*(1) FileScan csv [_c0#10497,X1#10498,X2#10499,X3#10500,X4#10501,X5#10502,X6#10503,X7#10504,X8#10505,X9#10506,X10#10507,X11#10508,X12#10509,X13#10510,X14#10511,X15#10512,X16#10513,X17#10514,X18#10515,X19#10516,X20#10517,X21#10518,X22#10519,X23#10520,Y#10521] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/GELab/Documents/Development/ML With Spark/Project/default of cre..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:string,X1:string,X2:string,X3:string,X4:string,X5:string,X6:string,X7:string,X8:string...


In [64]:
CreditDefaultData.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------+
|_c0|       X1| X2|       X3|      X4| X5|   X6|   X7|   X8|   X9|  X10|  X11|      X12|      X13|      X14|      X15|      X16|      X17|     X18|     X19|     X20|     X21|     X22|     X23|                   Y|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|default payment n...|
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|      

Above we observe that the 1st row is the actual header

#### Converting to pandas dataframe to set the first row as the header

In [65]:
CreditDefualtPdDf = CreditDefaultData.toPandas()

In [66]:
CreditDefualtPdDf.columns = CreditDefualtPdDf.loc[0,:]
CreditDefualtPdDf = CreditDefualtPdDf.loc[1:,:]

In [67]:
CreditDefualtPdDf.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


#### Converting back to spark dataframe

In [68]:
CreditDefaultData = spark.createDataFrame(CreditDefualtPdDf)

In [69]:
CreditDefaultData.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|default payment next month|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|                         1|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|  

#### Checking for columns having na values

In [70]:
CreditDefaultData.toPandas().isna().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64

### Statistics of the data

In [71]:
CreditDefaultData.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
ID,30000,15000.5,8660.398374208891,1,9999
LIMIT_BAL,30000,167484.32266666667,129747.66156720239,10000,90000
SEX,30000,1.6037333333333332,0.4891291960902606,1,2
EDUCATION,30000,1.8531333333333333,0.7903486597207294,0,6
MARRIAGE,30000,1.5518666666666667,0.5219696006132485,0,3
AGE,30000,35.4855,9.21790406809019,21,79
PAY_0,30000,-0.0167,1.1238015279973346,-1,8
PAY_2,30000,-0.13376666666666667,1.1971859730345529,-1,8
PAY_3,30000,-0.1662,1.1968675684465735,-1,8


In [72]:
CreditDefaultData.columns

['ID',
 'LIMIT_BAL',
 'SEX',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6',
 'default payment next month']

#### Casting all the features to Double and Label to Int

In [73]:
AllColumns = CreditDefaultData.columns


for colName in AllColumns:
    if colName != 'default payment next month':
        CreditDefaultData = CreditDefaultData.withColumn(colName, col(colName).cast("double"))
    if colName == 'default payment next month':
        CreditDefaultData = CreditDefaultData.withColumn(colName, col(colName).cast("int"))
    
    

In [74]:
CreditDefaultData.printSchema()

root
 |-- ID: double (nullable = true)
 |-- LIMIT_BAL: double (nullable = true)
 |-- SEX: double (nullable = true)
 |-- EDUCATION: double (nullable = true)
 |-- MARRIAGE: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- PAY_0: double (nullable = true)
 |-- PAY_2: double (nullable = true)
 |-- PAY_3: double (nullable = true)
 |-- PAY_4: double (nullable = true)
 |-- PAY_5: double (nullable = true)
 |-- PAY_6: double (nullable = true)
 |-- BILL_AMT1: double (nullable = true)
 |-- BILL_AMT2: double (nullable = true)
 |-- BILL_AMT3: double (nullable = true)
 |-- BILL_AMT4: double (nullable = true)
 |-- BILL_AMT5: double (nullable = true)
 |-- BILL_AMT6: double (nullable = true)
 |-- PAY_AMT1: double (nullable = true)
 |-- PAY_AMT2: double (nullable = true)
 |-- PAY_AMT3: double (nullable = true)
 |-- PAY_AMT4: double (nullable = true)
 |-- PAY_AMT5: double (nullable = true)
 |-- PAY_AMT6: double (nullable = true)
 |-- default payment next month: integer (nullable = true)



In [75]:
CreditDefaultData.toPandas().isna().sum()

ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64

##### Vecotr assembler

In [106]:
# encoder = OneHotEncoderEstimator(inputCols=inputs, outputCols=["categoryVec1", "categoryVec2"])
FeatureCols = AllColumns[1:-1] ## removing ID col and default payment next month

vectorAssembler = VectorAssembler(inputCols=FeatureCols, outputCol="features")
# .transform(CreditDefaultData).select('features','default payment next month')
# featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(CreditDefaultDataWithFeatures)

##### Feature Indexer

In [107]:
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)
stdScaler = StandardScaler(inputCol="indexedFeatures", outputCol="features_scaled")

### Pipeline

In [110]:
pipeline = Pipeline(stages=[vectorAssembler, featureIndexer,stdScaler])
pipelineModel = pipeline.fit(CreditDefaultData)

In [112]:
DataForTrain = pipelineModel.transform(CreditDefaultData).select('features_scaled','default payment next month')

In [117]:
DataForTrain = DataForTrain.withColumnRenamed("features_scaled", "features").withColumnRenamed("default payment next month", "label")

In [118]:
DataForTrain.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.15414535998894...|    1|
|[0.92487215993365...|    1|
|[0.69365411995024...|    0|
|[0.38536339997235...|    0|
|[0.38536339997235...|    0|
+--------------------+-----+
only showing top 5 rows



### Test train split

In [120]:
train, test = DataForTrain.randomSplit([0.7, 0.3], seed = 21)
print('numsaples in train : ',train.count(), 'numsamples in test : ', test.count())


numsaples in train :  20991 numsamples in test :  9009


#####  citation: https://docs.databricks.com/applications/machine-learning/mllib/binary-classification-mllib-pipelines.html
https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa

### Logistic Regression

In [128]:
LogisticClassifier = LogisticRegression(labelCol="label", featuresCol="features")
paramGrid = (ParamGridBuilder()\
             .addGrid(LogisticClassifier.regParam, [0.01, 0.5, 2.0])\
             .build())

crossval = CrossValidator(estimator=LogisticClassifier,estimatorParamMaps=paramGrid,numFolds=5,evaluator=BinaryClassificationEvaluator()) 
LogisticCVModel = crossval.fit(train)
predictions = LogisticCVModel.transform(test)

print('The area under roc curve (accuracy) of the model on the test data is:', BinaryClassificationEvaluator().evaluate(predictions))

The accuracy of the model on the test data is: 0.7189469016565526


#### Logistic model coefficientss are

In [135]:
weights = LogisticCVModel.bestModel.coefficients
weights = [(float(w),) for w in weights]  # convert numpy type to float, and to tuple
weightsDF = sqlContext.createDataFrame(weights, ["Feature Weight"])
weightsDF.show()

+--------------------+
|      Feature Weight|
+--------------------+
|-0.08275633595451475|
|-0.05998103517805089|
|-0.08397080208921334|
|-0.08014552753942009|
|  0.0750003175932271|
|  0.6156223015516452|
| 0.10331212746506263|
|  0.0918399231101285|
| 0.03862229026650423|
| 0.06199392693039379|
|-0.00785902846043...|
|-0.11826204240651074|
|-2.70031027617202...|
| 0.00925149497523386|
|0.006581679064127118|
|0.008717485276893994|
|0.001968844795337...|
|-0.15113516680086886|
|-0.11679192447993436|
|-0.05012471620652517|
+--------------------+
only showing top 20 rows



### Decision Tree

In [130]:
DTreeClassifier = DecisionTreeClassifier(labelCol="label", featuresCol="features")
paramGrid = (ParamGridBuilder()
             .addGrid(DTreeClassifier.maxDepth, [1, 2, 6, 10])
             .addGrid(DTreeClassifier.maxBins, [20, 40, 80])
             .build())

crossval = CrossValidator(estimator=DTreeClassifier,estimatorParamMaps=paramGrid,numFolds=5,evaluator=BinaryClassificationEvaluator()) 
DTreeCVModel = crossval.fit(train)
predictions = DTreeCVModel.transform(test)

print('The area under roc curve (accuracy) of the model on the test data is:', BinaryClassificationEvaluator().evaluate(predictions))

The accuracy of the model on the test data is: 0.4664263526078152


### Random Forest

In [137]:
rfClassifier = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
paramGrid = (ParamGridBuilder()\
             .addGrid(rfClassifier.maxDepth, [4, 8])
             .addGrid(rfClassifier.maxBins, [30, 60])
             .addGrid(rfClassifier.numTrees, [10,15,30])
             .build())

crossval = CrossValidator(estimator=rfClassifier,estimatorParamMaps=paramGrid,numFolds=5,evaluator=BinaryClassificationEvaluator()) 
rfCvModel = crossval.fit(train)
predictions = rfCvModel.transform(test)
print('The area under roc curve (accuracy) of the model on the test data is:', BinaryClassificationEvaluator().evaluate(predictions))

The area under roc curve (accuracy) of the model on the test data is: 0.7756961477172108


### Gradient Boositngs

In [141]:
bostingClassifier = GBTClassifier(labelCol="label", featuresCol="features", maxIter=20)
paramGrid = (ParamGridBuilder()
             .addGrid(bostingClassifier.maxDepth, [4, 8])
             .addGrid(bostingClassifier.maxBins, [30, 60])
             .build())
crossval = CrossValidator(estimator=bostingClassifier,estimatorParamMaps=paramGrid,numFolds=5,evaluator=BinaryClassificationEvaluator()) 
GBcvModel = crossval.fit(train)
predictions = GBcvModel.transform(test)
print('The area under roc curve (accuracy) of the model on the test data is:', BinaryClassificationEvaluator().evaluate(predictions))

The area under roc curve (accuracy) of the model on the test data is: 0.7759575305564212
